In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load data
print("Loading data...")
train_data = pd.read_csv('/kaggle/input/movie-recomendation-fall-2020/train.txt', sep='\t', header=None, names=['user_id', 'movie_id', 'rating'])

print(f"Dataset shape: {train_data.shape}")
print(f"Number of unique users: {train_data['user_id'].nunique()}")
print(f"Number of unique movies: {train_data['movie_id'].nunique()}")
print(f"Rating range: {train_data['rating'].min()} - {train_data['rating'].max()}")



Using device: cuda
Loading data...
Dataset shape: (90570, 3)
Number of unique users: 943
Number of unique movies: 1680
Rating range: 1 - 5


In [2]:
user_ids = sorted(train_data['user_id'].unique())  # unique(): Lấy ra danh sách các phần tử duy nhất trong cột; sorted():Sắp xếp theo thứ tự từ lớn ->bé
user_id_mapping = {id: i for i, id in enumerate(user_ids)} # Dictionary Chuyển id thành số theo thứ tự 
item_ids = sorted(train_data['movie_id'].unique())  # 1 to 1682
item_id_mapping = {id: i for i, id in enumerate(item_ids)}

train_data['user_id'] = train_data['user_id'].map(user_id_mapping) # Lúc này tất cả các id đều theo thứ tự và không bị mất dữ liệu thứ tự của user_id
train_data['movie_id'] = train_data['movie_id'].map(item_id_mapping)

num_users = len(user_ids)
num_items = len(item_ids)

print("Number of unique users:", num_users)
print("Number of unique items:", num_items)

Number of unique users: 943
Number of unique items: 1680


In [3]:
train_interactions = torch.tensor(train_data[['user_id', 'movie_id']].values, dtype=torch.long)
print(train_interactions.shape)
print(train_interactions[:10])
print("Data type:", train_interactions.dtype)
print("Device:", train_interactions.device)

torch.Size([90570, 2])
tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8],
        [0, 9]])
Data type: torch.int64
Device: cpu


In [4]:
# Adjacency matrix
rows = torch.cat([train_interactions[:, 0], train_interactions[:, 1] + num_users], dim=0) #Nối 2 cột user_id và movie_id để học tương tác
cols = torch.cat([train_interactions[:, 1] + num_users, train_interactions[:, 0]], dim=0)
# rows = [0, 0, 1, 4, 5, 3]  ## Các phần tử trong rows và cols là giống nhau, chỉ khác nhau về thứ tự để học các tương tác
# cols = [4, 5, 3, 0, 0, 1]
#          ↓   ↓  ↓  ↓  ↓  ↓
# Cạnh: (0,4), (0,5), (1,3), (4,0), (5,0), (3,1)
indices = torch.stack([rows, cols], dim=0).to(device)
values = torch.ones(indices.shape[1], device=device) #values :Tensor toàn số 1 và indices.shape[1] thể hiện cho số cạnh 
adj = torch.sparse_coo_tensor(indices, values, size=(num_users + num_items, num_users + num_items), device=device)
#adj :Ma trận kề biểu diễn tương tác giữa item và user
# Normalized adjacency matrix
degrees = torch.sparse.sum(adj, dim=1).to_dense() #Tính bậc của các node bằng các tính tổng từng hàng->degrees: mảng biểu diễn bậc của từng node
norm_values = 1.0 / (torch.sqrt(degrees[rows]) * torch.sqrt(degrees[cols])).to(device) #Tính hệ số cạnh giữa user và item
#A_norm[u, i] = 1 / (√degree(u) * √degree(i)):Phần tử đại diện cho gía trị cạnh trong ma trận norm_adj
norm_adj = torch.sparse_coo_tensor(indices, norm_values, size=(num_users + num_items, num_users + num_items), device=device) 


In [5]:
class SimGCL(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, num_layers, norm_adj, device):
        super(SimGCL, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.device = device
        self.register_buffer('norm_adj', norm_adj)
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)
        nn.init.normal_(self.user_embeddings.weight, std=0.01)  # Initialize user embeddings
        nn.init.normal_(self.item_embeddings.weight, std=0.01)  # Initialize item embeddings
        self.eps = 0.1 

    #Hàm này truyền thông tin qua các  layer GNN để tạo ra các Embedding cuối cùng cho user và item
    def forward(self, perturbed=False):
        # Concatenate initial user and item embeddings
        ego_embeddings = torch.cat([self.user_embeddings.weight, self.item_embeddings.weight], dim=0)
        #self.user_embedding: (num_user,embedding_dim); self.item_embedding: (num_item,embedding_dim)
        ## ---> ego_embeddings: (num_user+num_item,embedding_dim):Ghép 2 ma trận trên lại vs nhau
        #ego_embeddings chỉ thể hiện embedding của các node mà ko có sự tương tác  giữa các node 
        all_embeddings = []

        #Lặp qua số lượng các layer GNN
        for k in range(self.num_layers):
            
            ego_embeddings = torch.spmm(self.norm_adj, ego_embeddings) # Cho norm_adj(num_item+num_user;num_item+num_user)*(num_item+num_user;embeddings_dim)
           #ego_embeddings: lúc này là các embedding thể hiện đc đặc trưng của node(đã bao gồm vs sự tương tác của các node khác vs nó)

            #Nhiễu này không chỉ thêm 1 lần vào ban đầu mà thêm vào mỗi lớp GNN
            #Thêm nhiễu vào mô hình 
            if perturbed:
                #Ma trận nhiễm từ [0-1]
                random_noise = torch.rand_like(ego_embeddings).to(self.device)
                # Add normalized noise scaled by eps
                ego_embeddings += torch.sign(ego_embeddings) * F.normalize(random_noise, dim=-1) * self.eps

            #Qua mỗi lớp GNN, lưu lại toàn bộ embedding đại diện cho các node
            all_embeddings.append(ego_embeddings)

        # Stack embeddings across layers and compute the mean
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)

        # Split into user and item embeddings
        user_all_embeddings, item_all_embeddings = torch.split(
            all_embeddings, [self.num_users, self.num_items]) 
 # chia ma trận all_embeddings(num_user+item_user,embedding) thành 2 ma trận user_all_embedding(num_user,embedding)và item_all_embedding(item_user,embedding)

        return user_all_embeddings, item_all_embeddings

    def get_embeddings(self):
        all_embeddings = torch.cat([self.user_embeddings.weight, self.item_embeddings.weight], dim=0)
        ego_embeddings = all_embeddings
        for _ in range(self.num_layers):
            all_embeddings = torch.spmm(self.norm_adj, all_embeddings)
            ego_embeddings += all_embeddings
        final_embeddings = ego_embeddings / (self.num_layers + 1)
        user_emb = final_embeddings[:self.num_users]
        item_emb = final_embeddings[self.num_users:]
        return user_emb, item_emb

In [7]:
import torch.optim as optim
# Hyperparameters
embedding_dim = 64
num_layers = 3
learning_rate =1e-3
lambda_reg = 1e-6  # Regularization weight
cl_rate = 0.01  # Contrastive loss weight

# Initialize model
model = SimGCL(num_users, num_items, embedding_dim, num_layers, norm_adj, device)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
def InfoNCE(view1, view2, temperature: float, b_cos: bool = True):
    if b_cos:
        view1, view2 = F.normalize(view1, dim=1), F.normalize(view2, dim=1)
    pos_score = (view1 @ view2.T) / temperature
    score = torch.diag(F.log_softmax(pos_score, dim=1))
    return -score.mean()

def bpr_loss(user_emb, pos_item_emb, neg_item_emb):
    pos_score = (user_emb * pos_item_emb).sum(dim=1)
    neg_score = (user_emb * neg_item_emb).sum(dim=1)
    loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-8)
    return loss.mean()



In [13]:
def train_simgcl(
    model,
    train_interactions,
    num_epochs=50,
    batch_size=256,
    temperature=0.2,
    learning_rate=0.001,
    lambda_cl=0.1
):
    device = model.device
    model.train()

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_dataset = TensorDataset(
        train_interactions[:, 0],  # user
        train_interactions[:, 1]   # pos item
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_users, batch_pos_items in train_loader:
            batch_users = batch_users.to(device)
            batch_pos_items = batch_pos_items.to(device)

            # --------- Negative sampling ---------
            batch_neg_items = torch.randint(
                low=0,
                high=model.num_items,
                size=batch_pos_items.size(),
                device=device
            )

            optimizer.zero_grad()

            # ==================================================
            # 1. BPR LOSS (NO perturbation)
            # ==================================================
            user_emb, item_emb = model(perturbed=False)

            u_emb = user_emb[batch_users]
            pos_emb = item_emb[batch_pos_items]
            neg_emb = item_emb[batch_neg_items]

            loss_bpr = bpr_loss(u_emb, pos_emb, neg_emb)

            # ==================================================
            # 2. CONTRASTIVE LOSS (WITH perturbation)
            # ==================================================
            user_emb1, item_emb1 = model(perturbed=True)
            user_emb2, item_emb2 = model(perturbed=True)

            cl_user = InfoNCE(
                user_emb1[batch_users],
                user_emb2[batch_users],
                temperature
            )

            cl_item = InfoNCE(
                item_emb1[batch_pos_items],
                item_emb2[batch_pos_items],
                temperature
            )

            loss_cl = cl_user + cl_item

            # ==================================================
            # 3. JOINT LOSS
            # ==================================================
            loss = loss_bpr + lambda_cl * loss_cl

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(
            f"Epoch [{epoch+1}/{num_epochs}] | "
            f"Loss: {total_loss / len(train_loader):.4f} | "
            f"BPR: {loss_bpr.item():.4f} | CL: {loss_cl.item():.4f}"
        )

    return model


In [14]:
 train_interactions_tensor = torch.tensor(train_interactions, dtype=torch.long).to(device)

/tmp/ipykernel_55/2303990061.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_interactions_tensor = torch.tensor(train_interactions, dtype=torch.long).to(device)


In [15]:
from torch.utils.data import TensorDataset, DataLoader
trained_model = train_simgcl(
        model=model,
        train_interactions=train_interactions_tensor,
        num_epochs=50,
        batch_size=256,
        temperature=0.2,
        learning_rate=0.001
    )

Epoch [1/50] | Loss: 0.9398 | BPR: 0.6865 | CL: 2.1589
Epoch [2/50] | Loss: 0.9385 | BPR: 0.6856 | CL: 2.1898
Epoch [3/50] | Loss: 0.9376 | BPR: 0.6842 | CL: 2.2012
Epoch [4/50] | Loss: 0.9366 | BPR: 0.6814 | CL: 2.1593
Epoch [5/50] | Loss: 0.9357 | BPR: 0.6824 | CL: 2.1644
Epoch [6/50] | Loss: 0.9342 | BPR: 0.6802 | CL: 2.2465
Epoch [7/50] | Loss: 0.9327 | BPR: 0.6794 | CL: 2.2315
Epoch [8/50] | Loss: 0.9312 | BPR: 0.6802 | CL: 2.2227
Epoch [9/50] | Loss: 0.9299 | BPR: 0.6694 | CL: 2.2504
Epoch [10/50] | Loss: 0.9280 | BPR: 0.6673 | CL: 2.2031
Epoch [11/50] | Loss: 0.9261 | BPR: 0.6735 | CL: 2.1967
Epoch [12/50] | Loss: 0.9242 | BPR: 0.6688 | CL: 2.2405
Epoch [13/50] | Loss: 0.9218 | BPR: 0.6696 | CL: 2.2431
Epoch [14/50] | Loss: 0.9196 | BPR: 0.6493 | CL: 2.2510
Epoch [15/50] | Loss: 0.9170 | BPR: 0.6351 | CL: 2.3024
Epoch [16/50] | Loss: 0.9145 | BPR: 0.6655 | CL: 2.2218
Epoch [17/50] | Loss: 0.9116 | BPR: 0.6545 | CL: 2.2363
Epoch [18/50] | Loss: 0.9082 | BPR: 0.6685 | CL: 2.2793
E

In [16]:
import pandas as pd
import numpy as np
import torch

# 1. Tải lại tập test với đúng định dạng
test_df = pd.read_csv('/kaggle/input/movie-recomendation-fall-2020/test.txt', 
                     sep='\t', 
                     header=None,
                     names=['user_id', 'movie_id'])

print("Test data shape:", test_df.shape)
print("Test data columns:", test_df.columns.tolist())
print("\nFirst few rows:")
print(test_df.head())
print("\nDữ liệu test:")
print(test_df.info())

# 2. Tạo cột Id cho file submit (từ 1 đến số lượng mẫu)
test_df['Id'] = range(1, len(test_df) + 1)

# 3. Ánh xạ user_id và movie_id sang index đã chuẩn hóa
test_df['user_id_mapped'] = test_df['user_id'].map(user_id_mapping)
test_df['movie_id_mapped'] = test_df['movie_id'].map(item_id_mapping)

# Đếm số user/item không có trong tập train
missing_users = test_df['user_id_mapped'].isna().sum()
missing_items = test_df['movie_id_mapped'].isna().sum()
print(f"\nSố user không có trong tập train: {missing_users}/{len(test_df)}")
print(f"Số item không có trong tập train: {missing_items}/{len(test_df)}")

# 4. Đảm bảo model ở chế độ đánh giá
trained_model.eval()

# 5. Lấy embeddings từ model
with torch.no_grad():
    user_emb, item_emb = trained_model.get_embeddings()

# 6. Tạo dự đoán
predictions = []

for _, row in test_df.iterrows():
    user_idx = row['user_id_mapped']
    item_idx = row['movie_id_mapped']
    
    # Kiểm tra xem user/item có trong tập train không
    if pd.isna(user_idx) or pd.isna(item_idx):
        # Nếu user hoặc item mới, dùng rating trung bình = 2.5
        pred_rating = 2.5
    else:
        # Chuyển sang integer
        user_idx = int(user_idx)
        item_idx = int(item_idx)
        
        # Lấy embeddings
        user_embedding = user_emb[user_idx]
        item_embedding = item_emb[item_idx]
        
        # Tính rating dự đoán bằng dot product
        pred_rating = torch.dot(user_embedding, item_embedding).item()
        
        # Chuẩn hóa rating về khoảng [1, 5]
        # Cách 1: Dùng sigmoid function
        pred_rating = 1 + 4 * (1 / (1 + np.exp(-pred_rating)))
        
        # Cách 2: Hoặc scale dựa trên min-max (cần tính từ train)
        # Giả sử dot product trong khoảng [-10, 10]
        # pred_rating = (pred_rating + 10) / 20 * 4 + 1
        
        # Đảm bảo trong khoảng [1, 5]
        pred_rating = max(1.0, min(5.0, pred_rating))
    
    predictions.append(pred_rating)

# 7. Tạo file submit
test_df['Score'] = predictions

# Chỉ lấy 2 cột cần thiết cho submission
submit_df = test_df[['Id', 'Score']].copy()

# 8. Lưu file submit
submit_path = '/kaggle/working/SimGCL2.csv'
submit_df.to_csv(submit_path, index=False)

print(f"\n✅ File submit đã được tạo tại: {submit_path}")
print(f"Kích thước file: {submit_df.shape}")
print(f"\n10 dòng đầu tiên của file submit:")
print(submit_df.head(10))
print(f"\nThống kê rating dự đoán:")
print(submit_df['Score'].describe())

# 9. Kiểm tra định dạng file
print(f"\n📋 Kiểm tra file submit:")
sample = pd.read_csv(submit_path)
print(f"Cột: {sample.columns.tolist()}")
print(f"Kiểu dữ liệu: {sample.dtypes.tolist()}")
print(f"Min Score: {sample['Score'].min():.4f}")
print(f"Max Score: {sample['Score'].max():.4f}")
print(f"Mean Score: {sample['Score'].mean():.4f}")

# 10. Lưu thêm bản backup để kiểm tra
backup_path = '/kaggle/working/submission_with_details.csv'
test_df[['Id', 'user_id', 'movie_id', 'user_id_mapped', 'movie_id_mapped', 'Score']].to_csv(backup_path, index=False)
print(f"\n📁 File backup chi tiết: {backup_path}")

Test data shape: (9430, 2)
Test data columns: ['user_id', 'movie_id']

First few rows:
   user_id  movie_id
0        1        20
1        1        33
2        1        61
3        1       117
4        1       155

Dữ liệu test:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9430 entries, 0 to 9429
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   user_id   9430 non-null   int64
 1   movie_id  9430 non-null   int64
dtypes: int64(2)
memory usage: 147.5 KB
None

Số user không có trong tập train: 0/9430
Số item không có trong tập train: 2/9430

✅ File submit đã được tạo tại: /kaggle/working/SimGCL2.csv
Kích thước file: (9430, 2)

10 dòng đầu tiên của file submit:
   Id     Score
0   1  4.999327
1   2  1.110645
2   3  3.881274
3   4  3.088794
4   5  4.762532
5   6  3.679695
6   7  4.930430
7   8  4.987152
8   9  1.886794
9  10  1.696910

Thống kê rating dự đoán:
count    9430.000000
mean        4.097367
std         1.298602
min